# Prepare data in the format for modelling

In [64]:
import pandas as pd
from warnings import simplefilter
import sys
simplefilter(action='ignore', category=FutureWarning)
import geopy.distance

def add_sin_and_cos_features(df, column_to_transform):
    df['Sin_{}'.format(column_to_transform)] = np.sin(2 * np.pi * df[column_to_transform] / max(df[column_to_transform])) 
    df['Cos_{}'.format(column_to_transform)] = np.cos(2 * np.pi * df[column_to_transform] / max(df[column_to_transform]))
    return df

sys.path.append('../../3. Modelling')
from Functions import *

In [65]:
buffer_size_m = 400

### Prepare footfall data
<u> Including removing outliers:</u>  
The model should predict normal footfall. Therefore any days that have extremely high or low footfall should be taken out of the training data. We don't actually want the model to try to predict footfall on unusual days, because the things that make the day unusual (like errors in the camera counters, or the presence of special events) are not captured in the input data.

Outliers are detected using the Median Absolute Deviation method.

In [114]:
# Read in data
sensors = pd.read_csv('../../Cleaned_data/SensorData/allsensors.csv')
# Create month as number not string
sensors['datetime'] =pd.to_datetime(sensors['datetime'], format = '%Y-%m-%d %H:%M:%S')#dayfirst = False)
# Keep only data from 2011 onwards
sensors= sensors[sensors['year']>2010]
# # Create a categrorial variable defining the time of day
sensors['time_of_day'] = sensors.apply (lambda row: label_hours(row), axis=1)

In [ ]:
# Remove outliers
sensors, outliers = remove_outliers(sensors)
# Drop unneeded columns
sensors=sensors.drop(['Latitude', 'Longitude', 'location', 'mdate'], axis=1)
# Check the data
sensors.head()

### Inspect outliers which have been removed

In [129]:
# # Look at outliers from particular day (which have reason to believe might have incldued outliers)
# t = [pd.to_datetime('2020-03-07 13:00:00')]
# this_day = outliers[outliers['datetime'].isin(t)]#[2255323]['datetime']
# # Look at biggest outliers
# outliers.nlargest(10, 'hourly_counts')

### Inspecting sensor which has a lot of the outliers removed

In [ ]:
# sensor_57 = sensors[sensors['sensor_id']==57]
# # Filter by a single date
# sensor_57_oneday = sensor_57[sensor_57['datetime'].dt.strftime('%Y-%m-%d') == "2019-11-15"]
# sensor_57_oneday.index=sensor_57_oneday['time']
# fig, ax = plt.subplots(figsize = (5,4), sharex = True)
# fig = sensor_57_oneday['hourly_counts'].plot(ax=ax, color='darkred', linewidth=2, legend =True)

## add date variables

In [4]:
sensors['day_of_month_num'] = sensors['datetime'].dt.day
sensors['weekday_num'] = sensors['datetime'].dt.weekday +1
sensors['month_num'] = sensors['datetime'].dt.month

### Join public holiday and weather data to sensor data (WHY DO NUMBER OF ROWS INCREASE SLIGHTLY?)

In [5]:
sensors_with_features = sensors.merge(pd.read_csv('../../Cleaned_data/WeatherData/weather_data_allyears.csv', parse_dates=['datetime']), on='datetime', how='left')
sensors_with_features = sensors_with_features.merge(pd.read_csv('../../Cleaned_data/WeatherData/DailyRainfallData.csv', parse_dates=['datetime']), on='datetime', how='left')
sensors_with_features = sensors_with_features.merge(pd.read_csv('../../Cleaned_data/HolidaysData/publicholidays.csv', parse_dates=['datetime']),how='left', on='datetime')
sensors_with_features = sensors_with_features.merge(pd.read_csv('../../Cleaned_data/HolidaysData/schoolholidays.csv', parse_dates=['datetime']),how='left', on='datetime')
sensors_with_features = sensors_with_features.merge(pd.read_csv('../../Cleaned_data/SpatialFeatures/sensors_betweenness.csv'),how='left', on='sensor_id')
sensors_with_features = sensors_with_features.merge(pd.read_csv('../../Cleaned_data/FeaturesNearSensors/num_features_near_sensors_{}.csv'.format(buffer_size_m), index_col=0) ,how='left', on='sensor_id')
sensors_with_features = sensors_with_features.merge(pd.read_csv('../../Cleaned_data/FeaturesNearSensors/feature_subtypes_near_sensors_{}.csv'.format(buffer_size_m), index_col=0) ,how='left', on='sensor_id')

In [6]:
print("Length is now: " , len(sensors_with_features))

Length is now:  4129082


### Add average number of floors of building in vicinity, for correct year 

In [ ]:
sensors_with_features['avg_n_floors'] = sensors_with_features.apply (lambda row: select_n_floors(row), axis=1)
sensors_with_features = sensors_with_features[sensors_with_features.columns.drop(list(sensors_with_features.filter(regex='avg_n_floors_')))]

### Add buildings (correctly for the year the data relates to)

In [ ]:
# FOR SENSORS WITH FEATURES
sensors_with_features['buildings'] = sensors_with_features.apply (lambda row: select_buildings(row), axis=1).copy()
sensors_with_features= sensors_with_features.drop(['buildings_2010', 'buildings_2011','buildings_2012', 'buildings_2013',
                                                  'buildings_2014','buildings_2015','buildings_2016','buildings_2017',
                                                  'buildings_2018', 'buildings_2019', 'buildings_2020'], axis =1)
# FOR SENSORS WITH SUBFEATURES
# Create a dataframe containing just the building subttypes for the year that this row refers to
temp = pd.DataFrame(None)
# For each year, get the data for just that year
for year in range(2011,2022+1):   
    # Get just footfall data for this year
    this_year = sensors_with_features[sensors_with_features['year'] == year]
    # If year is over 2020 then set the year to 2020 for the purposes of selecting the building data 
    if year > 2020:
        year = 2020
    # Get just the building columns for this year
    buildings_this_yr = this_year.filter(like='{}'.format(year))
    # Drop all the building subtype columns from the row (and the bikes) 
    this_year = this_year[this_year.columns.drop(list(this_year.filter(regex='bikes|buildings_')))]
     # Join the row without any buildings, back to this row 
    this_year = pd.concat([this_year, buildings_this_yr], axis=1)
    # Rename -- ??
    this_year.columns = this_year.columns.str.replace(r'_{}'.format(year), '')
    # Join to dataframe which will store data for all years eventually
    temp = temp.append(this_year)
sensors_with_features = temp.copy()

### Add dummy variables for calendar variables 
(Not doing this anymore as creating cos and sin variables)

In [ ]:
# sensors_with_features=convert_df_variables_to_dummy(sensors_with_features, ['day', 'month', 'year', 'time'])
# sensors_with_features=convert_df_variables_to_dummy(sensors_with_features, ['time_of_day'])
# sensors_with_features = sensors_with_features.drop(['datetime'],axis=1)

### Date based variables: Option 2 - Create Dummy Variables
for date_col in ['day', 'month',]:
    date_col_dummy =  pd.get_dummies(sensors_with_features[date_col], drop_first = True)
    if date_col =='month':
        date_col_dummy.columns= prepend(date_col_dummy.columns.values, 'month_')
    # if date_col =='year':
        # date_col_dummy.columns= prepend(date_col_dummy.columns.values, 'year_')
    sensors_with_features = pd.concat([sensors_with_features, date_col_dummy],axis=1)
    del sensors_with_features[date_col]

### Create sin/cos variables to represent cyclical features

In [ ]:
# Time refers to the hour of the day 0-23 (makes sense that this is cyclical - relationship between 23 and 0)
sensors_with_features = add_sin_and_cos_features(sensors_with_features, 'time')
# Month number from 1-12 (makes sense that this is cyclical - relationship between 12 and 1)
sensors_with_features = add_sin_and_cos_features(sensors_with_features, 'month_num')
# Weekday number from 1-7 (maybe doesnt make sense that this is cyclical - doesn't really follow a logical pattern?
sensors_with_features = add_sin_and_cos_features(sensors_with_features, 'weekday_num')

In [ ]:
ax = sensors_with_features.plot.scatter('Sin_weekday_num', 'Cos_weekday_num').set_aspect('equal')

### Replace NaNs with 0s

In [ ]:
sensors_with_features= sensors_with_features.fillna(0)

## Create aggregated data

In [ ]:
# ### Filter to include just sensors which we know have quite complete data 
# data = sensors_with_features[sensors_with_features['sensor_id'].isin([2,6,9,10,14,18])]
# data.reset_index(inplace=True, drop = True)

# # Get just this hourly counts
# hourly_counts = data[['datetime', "hourly_counts"]]
# # Get the sum of all values for one hour
# summed_hourly_counts = hourly_counts.groupby("datetime").sum()
# # reset index (for joining)
# summed_hourly_counts.reset_index(inplace = True, drop = True)

# # Get just the features
# features = data[['datetime','Temp', 'Humidity', 'Pressure', 'Rain', 'WindSpeed',
#        'public_holiday','school_holiday','Sin_time', 'Cos_time', 'Sin_month_num', 'Cos_month_num', 'Sin_weekday_num', 'Cos_weekday_num']]
# # Keep only one version of each row (should all be the same)
# features = features.drop_duplicates(keep='last')
# # reset index (for joining)
# features.reset_index(inplace = True, drop = True)

# # Join features to aggregated count values
# aggregated = pd.concat([summed_hourly_counts, features], axis=1, join='inner')

In [ ]:
# for column in sensors_with_features.columns:
#     print(column, np.nanmax(sensors_with_features[column]))

### Remove time variables no longer required

In [ ]:
# sensors_with_features = sensors_with_features.drop([ 'Sin_month_num', 'Cos_month_num', 'Sin_weekday_num',
#        'Cos_weekday_num',],axis=1)

### Add distance from centre as a variable

In [ ]:
### Read in Melbourne sensor location spatial data
melbourne_sensors = pd.read_csv("../../Data/FootfallData/melbourne_locations.csv")
melbourne_sensors.rename(columns={'sensor_description': 'Name'}, inplace = True)
melbourne_sensors = melbourne_sensors.drop_duplicates('sensor_id', keep='first')

# Coordinates of 'centre' of CBD (done on google maps)
coords_1 = (-37.812187461761596, 144.962265054567)
distances =[]
for row_number in range(0,len(melbourne_sensors)):
    coords_2 = (melbourne_sensors['Latitude'][row_number], melbourne_sensors['Longitude'][row_number])
    distances.append(geopy.distance.geodesic(coords_1, coords_2).km)
    
melbourne_sensors['distance_from_centre']=distances
melbourne_sensors=melbourne_sensors[['sensor_id','distance_from_centre']]

In [ ]:
sensors_with_features= pd.merge(sensors_with_features, melbourne_sensors, on=['sensor_id'], how='left')

### Save as csv

In [ ]:
sensors_with_features.to_csv("../../Cleaned_data/FormattedDataForModelling/formatted_data_for_modelling_allsensors_{}.csv".format(buffer_size_m),
                            index = False)

In [ ]:
sensors_with_features

In [ ]:
def doubleMADsfromMedian(y,thresh=3.5):
    """Find outliers using the Median Average Distance.
    
    VALUE: return a list of true/false denoting whether the element in y is an outlier or not
    
    PARAMETERS:
      - y is a pandas Series, or something like that.
    
    warning: this function does not check for NAs
    nor does it address issues when 
    more than 50% of your data have identical values
    """
    # Calculate the upper and lower limits
    m = np.median(y) # The median
    abs_dev = np.abs(y - m) # The absolute difference between each y and the median
    # The upper and lower limits are the median of the difference
    # of each data point from the median of the data
    left_mad = np.median(abs_dev[y <= m]) # The left limit (median of lower half)
    right_mad = np.median(abs_dev[y >= m]) # The right limit (median of upper half)
    
    # Now create an array where each value has left_mad if it is in the lower half of the data,
    # or right_mad if it is in the upper half
    y_mad = left_mad * np.ones(len(y)) # Initially every value is 'left_mad'
    y_mad[y > m] = right_mad # Now larger values are right_mad

    # Calculate the z scores for each element
    modified_z_score = 0.6745 * abs_dev / y_mad
    modified_z_score[y == m] = 0
    
    # Return boolean list showing whether each y is an outlier
    return modified_z_score > thresh

# Make a list of true/false for whether the footfall is an outlier
# no_outliers = pd.DataFrame(doubleMADsfromMedian(sensors_with_features['hourly_counts']))
# no_outliers.columns = ['outlier'] # Rename the column to 'outlier'

# # Join to the original footfall data to the list of outliers, then select a few useful columns
join = pd.concat([sensors_with_features, no_outliers], axis = 1)
# join = pd.DataFrame(join, columns = ['Day_yr', 'outlier', 'InCount'])
join
# # Choose just the outliers
outliers = join[join['outlier'] == True]
outliers_list = list(outliers['datitime']) # A list of the days that are outliers
outliers_list
# # Now remove all outliers from the original data
# df = original.loc[~original['Day_yr'].isin(outliers_list)]
# df = df.reset_index(drop = True)

# # Check that the lengths all make sense
# assert(len(df) == len(original)-len(outliers_list))

# print("I found {} outliers from {} days in total. Removing them leaves us with {} events".format(\
#     len(outliers_list), len(join), len(df) ) )